## **Módulo de Vigilancia y Priorización OpenSky**

### **Importación de librerías**

In [22]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import time

In [31]:
# ==============================================================================
# 1. CONFIGURACIÓN DEL RADAR (ADS-B)
# ==============================================================================
URL_OPENSKY = "https://opensky-network.org/api/states/all"
HEADERS = {'User-Agent': 'AviationDataPipeline_Etapa2'}

# Bounding Box: Cobertura Total Perú para el escaneo nacional
BOUNDS_PERU = {"lamin": -18.5, "lamax": 0.0, "lomin": -81.5, "lomax": -68.5}

# ==============================================================================
# 2. FUNCIÓN DE EXTRACCIÓN Y FILTRADO GEORREFERENCIADO
# ==============================================================================
def ejecutar_etapa_2_radar(lat_e1, lon_e1, radio_operativo=150):
    """
    Recibe coordenadas de la Etapa 1 y escanea el radar OpenSky.
    """
    session = requests.Session()
    try:
        print(f"📡 [ETAPA 2] Iniciando escaneo de radar... ({datetime.now().strftime('%H:%M:%S')})")
        
        # Ingesta de datos geofenced para Perú
        res = session.get(URL_OPENSKY, params=BOUNDS_PERU, headers=HEADERS, timeout=20)
        res.raise_for_status()
        vuelos_raw = res.json().get("states", [])

        if not vuelos_raw:
            return "📭 Radar nacional sin vuelos reportados."

        # Procesamiento y Normalización
        df = pd.DataFrame(vuelos_raw)[[0, 1, 2, 5, 6, 7, 9, 11]]
        df.columns = ['icao24', 'callsign', 'pais', 'lon', 'lat', 'alt', 'vel', 'v_rate']
        df['callsign'] = df['callsign'].str.strip()

        # Cálculo de proximidad respecto a la Etapa 1
        df['dist_km'] = np.sqrt((df['lat'] - lat_e1)**2 + (df['lon'] - lon_e1)**2) * 111.12
        
        # Filtro de zona de influencia
        vuelos_cerca = df[df['dist_km'] <= radio_operativo].copy()

        if vuelos_cerca.empty:
            # Lógica de respaldo: Identificar el tráfico más próximo en el país
            vuelo_proximo = df.sort_values('dist_km').iloc[0]
            return f"🟢 [LIMA] Espacio aéreo local despejado. Tráfico más cercano: {vuelo_proximo['callsign']} a {round(vuelo_proximo['dist_km'], 1)} km."

        # Inteligencia de Alerta (Basada en Altitud y Descenso)
        def clasificar_riesgo(row):
            if row['alt'] < 5000 and row['v_rate'] < 0: return "🔴 CRÍTICO (Aproximación)"
            if row['alt'] < 8000 and row['v_rate'] < 0: return "🟡 ADVERTENCIA (Descenso)"
            return "🟢 CRUCERO"

        vuelos_cerca['ESTADO'] = vuelos_cerca.apply(clasificar_riesgo, axis=1)
        
        return vuelos_cerca[['callsign', 'alt', 'vel', 'dist_km', 'ESTADO']].sort_values('dist_km')

    except Exception as e:
        return f"❌ Error en Etapa 2: {e}"

# ==============================================================================
# 3. EJECUCIÓN (SIMULANDO RECIBIR DATOS DE ETAPA 1)
# ==============================================================================
if __name__ == "__main__":
    # Datos recibidos de la Etapa 1 (Visual Crossing)
    LAT_LIMA, LON_LIMA = -12.02, -77.11
    
    reporte_radar = ejecutar_etapa_2_radar(LAT_LIMA, LON_LIMA)

    print("\n" + "="*90)
    print("📊 RESULTADO DE LA ETAPA 2: MONITOREO ADS-B (OPEN SKY)")
    print("="*90)
    
    if isinstance(reporte_radar, pd.DataFrame):
        print(reporte_radar.to_string(index=False))
        reporte_radar.to_csv("TELEMETRIA_ETAPA2.csv", index=False)
    else:
        print(reporte_radar)
        
    print("="*90)

📡 [ETAPA 2] Iniciando escaneo de radar... (14:45:14)

📊 RESULTADO DE LA ETAPA 2: MONITOREO ADS-B (OPEN SKY)
🟢 [LIMA] Espacio aéreo local despejado. Tráfico más cercano: LPE2234 a 337.4 km.


### **Dashboard Final**

In [33]:
class DashboardRadarPeru:
    """
    Sistema Central de Vigilancia ADS-B - Etapa 2.
    Especializado en detección de proximidad y análisis cinemático de aeronaves.
    """
    def __init__(self):
        self.url = "https://opensky-network.org/api/states/all"
        # Bounding Box: Cobertura Total Perú
        self.bounds = {"lamin": -18.5, "lamax": 0.0, "lomin": -81.5, "lomax": -68.5}
        self.headers = {'User-Agent': 'AviationDataPipeline_Etapa2_Pro'}
        # Puntos de Control Estratégicos
        self.puntos_control = [
            {"ID": "SPJC", "NOM": "LIMA", "LAT": -12.02, "LON": -77.11},
            {"ID": "SPZO", "NOM": "CUSCO", "LAT": -13.53, "LON": -71.93},
            {"ID": "SPQU", "NOM": "AREQUIPA", "LAT": -16.34, "LON": -71.56},
            {"ID": "SPQT", "NOM": "IQUITOS", "LAT": -3.78, "LON": -73.30}
        ]

    def capturar_telemetria(self):
        """Consume la API de OpenSky y normaliza los datos ADS-B."""
        try:
            print(f"📡 ESCANEANDO RADAR NACIONAL... ({datetime.now().strftime('%H:%M:%S')})")
            res = requests.get(self.url, params=self.bounds, headers=self.headers, timeout=20)
            res.raise_for_status()
            states = res.json().get("states", [])

            if not states:
                return pd.DataFrame()

            # Mapeo profesional: icao24, callsign, pais, lon, lat, altitud, velocidad, tasa_vertical
            df = pd.DataFrame(states)[[0, 1, 2, 5, 6, 7, 9, 11]]
            df.columns = ['icao24', 'callsign', 'pais', 'lon', 'lat', 'alt', 'vel', 'v_rate']
            df['callsign'] = df['callsign'].str.strip()
            return df
        except Exception as e:
            print(f"❌ Error de red en Etapa 2: {e}")
            return pd.DataFrame()

    def procesar_alerta(self, row):
        """Lógica de Inteligencia de Radar para determinar el riesgo operativo."""
        alt = row['alt'] if row['alt'] else 0
        v_rate = row['v_rate'] if row['v_rate'] else 0
        
        # Criterios de Aeronavegabilidad
        if alt < 5000 and v_rate < 0: return "🔴 CRÍTICO (Aproximación)"
        if v_rate < -1: return "🟡 ADVERTENCIA (Descenso)"
        if alt < 5000 and v_rate > 0: return "🔵 INFO (Despegue)"
        return "🟢 SEGURO (Crucero)"

    def generar_reporte(self):
        df_radar = self.capturar_telemetria()
        if df_radar.empty:
            print("📭 Radar despejado: Sin tráfico en territorio nacional.")
            return

        reporte_final = []

        for punto in self.puntos_control:
            # Cálculo de distancia geodésica al punto de control
            df_radar['dist'] = np.sqrt((df_radar['lat'] - punto['LAT'])**2 + 
                                       (df_radar['lon'] - punto['LON'])**2) * 111.12
            
            # Filtrado por radio de acción operativo (150 km)
            cercanos = df_radar[df_radar['dist'] <= 150].copy()
            
            if cercanos.empty:
                # Si no hay vuelos cerca, identificamos el más próximo en el país
                v_proximo = df_radar.sort_values('dist').iloc[0]
                reporte_final.append({
                    "ZONA": punto['NOM'], "VUELOS": 0, "STATUS": "🟢 DESPEJADO",
                    "OBJETIVO_CRÍTICO": f"Ninguno (Próximo: {v_proximo['callsign']} a {round(v_proximo['dist'], 1)}km)"
                })
            else:
                # Análisis cinemático de los vuelos en radio
                cercanos['ALERTA'] = cercanos.apply(self.procesar_alerta, axis=1)
                
                # Determinación del estatus de la zona
                num = len(cercanos)
                criticos = len(cercanos[cercanos['ALERTA'].str.contains("🔴")])
                
                status = "🟢 OPERATIVO"
                if criticos > 0 or num > 4: status = "🔴 SATURADO / RIESGO"
                elif num > 2: status = "🟡 TRÁFICO MODERADO"

                # Identificar el vuelo con mayor prioridad (el más cercano)
                v_top = cercanos.sort_values('dist').iloc[0]
                
                # Cálculo de ETA estimado (Minutos)
                # d = v * t -> t = d / v (vel en m/s a km/min)
                eta = round(v_top['dist'] / (v_top['vel'] * 0.06), 1) if v_top['vel'] > 0 else "N/A"

                reporte_final.append({
                    "ZONA": punto['NOM'], "VUELOS": num, "STATUS": status,
                    "OBJETIVO_CRÍTICO": f"{v_top['callsign']} (ETA: {eta} min)"
                })

        # --- VISUALIZACIÓN DE TABLA FINAL ---
        df_final = pd.DataFrame(reporte_final)
        print("\n" + "="*115)
        print(f"📊 PANEL DE CONTROL DE TRÁFICO AÉREO - ETAPA 2 (OPEN SKY)")
        print("="*115)
        print(df_final.to_string(index=False, justify='center'))
        print("="*115)
        
        # Persistencia de datos exclusiva
        df_final.to_csv("REPORT_ETAPA2_OPENSKY.csv", index=False)
        print(f"📁 Reporte de telemetría generado: 'REPORT_ETAPA2_OPENSKY.csv'")

if __name__ == "__main__":
    radar = DashboardRadarPeru()
    radar.generar_reporte()

📡 ESCANEANDO RADAR NACIONAL... (14:45:16)

📊 PANEL DE CONTROL DE TRÁFICO AÉREO - ETAPA 2 (OPEN SKY)
  ZONA    VUELOS    STATUS             OBJETIVO_CRÍTICO          
    LIMA    0    🟢 DESPEJADO Ninguno (Próximo: LPE2234 a 337.9km)
   CUSCO    0    🟢 DESPEJADO Ninguno (Próximo: LPE2234 a 656.8km)
AREQUIPA    0    🟢 DESPEJADO Ninguno (Próximo: LPE2234 a 931.4km)
 IQUITOS    0    🟢 DESPEJADO  Ninguno (Próximo: CMP125 a 216.2km)
📁 Reporte de telemetría generado: 'REPORT_ETAPA2_OPENSKY.csv'


### **Conclusión**

"El Módulo de la Etapa 2 (OpenSky) se ha diseñado como una unidad de vigilancia autónoma. Utiliza telemetría ADS-B para monitorear la densidad del tráfico aéreo nacional y clasificar la prioridad de las aeronaves mediante un análisis de sus vectores de altitud y velocidad, operando de manera independiente a las validaciones de tierra o pronósticos externos.".